In [1]:
import math 
import operator

class SimpleLogisticRegression(object):
    def __init__(self, features_num):
        self._m_features_num = features_num
        self._m_weights = [1.0 / self._m_features_num] * self._m_features_num
        self._m_bias = 1.0
        
        self._m_best_weights = [0.0] * self._m_features_num
        self._m_best_bias = 0.0
    
    def _sigmoid(self, w, x, b):
        return 1.0 / (1.0 + math.exp(-sum(map(operator.mul, w, x)) - b))
    
    def _likelihood(self, pred_y, y):
        likelihood = 0.0
        sample_num = 0
        for py, ty in zip(pred_y, y):
            likelihood += ty * math.log(py) + (1 - ty) * math.log(1 - py)
            sample_num += 1
        return likelihood / sample_num
    
    def fit(self, features, labels, learning_rate=0.1, epoches=10000):
        sample_num = len(features)
        best_likelihood, best_epoch = None, None
        for epoch in range(epoches):
            # 计算梯度
            gradient = [0.] * (self._m_features_num + 1)
            for tx, ty in zip(features, labels):
                delta = ty - self._sigmoid(self._m_weights, tx, self._m_bias)
                for i, xi in enumerate(tx):
                    gradient[i] += delta * xi
                gradient[-1] += delta
            
            # 更新参数
            for i, grad in enumerate(gradient[:-1]):
                self._m_weights[i] += learning_rate * grad / sample_num
            self._m_bias += learning_rate * gradient[-1] / sample_num
            
            # 评估新模型
            pred_y = [self._sigmoid(self._m_weights, x, self._m_bias) for x in features]
            likelihood = self._likelihood(pred_y, labels)
            if best_likelihood is None or likelihood - best_likelihood > 1e-8:
                best_likelihood = likelihood
                best_epoch = epoch
                self._m_best_weights = self._m_weights[:]
                self._m_best_bias = self._m_bias
            elif epoch - best_epoch >= 10:
                break;
    
    def predict(self, features):
        return [self._sigmoid(self._m_best_weights, x, self._m_bias) for x in features]

In [2]:
lr = SimpleLogisticRegression(3)
features = [[1, 2, 3], [2, 4, 6], [3, 5, 7], [4, 6, 8]]
labels = [0, 0, 1, 1]
lr.fit(features, labels)
print(lr.predict(features))

[0.004497222342978742, 0.018361156480988333, 0.983003700051452, 0.9999944082938986]
